# Feedback System Interface
June 7th, 2022<br>
Adrienne Ko (Adrienne.Ko.23@dartmouth.edu)<br>
Jack Keane (John.F.Keane.22@dartmouth.edu)

The following code takes in an audio file (.wav) and a target gender, and outputs a prediction of the speaker’s gender identity along with a suggestion on how to improve one’s voice if the target and predicted genders do not match.

In [3]:
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestClassifier

import shap

import librosa
from scipy.io import wavfile

from extractors import get_pitch_intonation, get_formant_diffs

In [13]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
df = pd.read_csv('./audio_features_final.csv', index_col=0)
df['resonance'] = df['f1_diff'] + df['f2_diff']

X = df[['pitch', 'intonation', 'resonance']]
y = df['gender'] #.apply(lambda x: int(x == 'male'))

In [5]:
rf = RandomForestClassifier(criterion='entropy', max_features=2, min_samples_leaf=5)
rf.fit(X, y)

RandomForestClassifier(criterion='entropy', max_features=2, min_samples_leaf=5)

In [6]:
explainer = shap.TreeExplainer(rf)

In [17]:
testfile = './audio_data/mcgill/FA/FA01_02.wav'

In [ ]:
while True:
    target_gender = ''
    while not target_gender:
        user_input = input('Enter your target gender (male, female): ')
        if user_input.lower() in {'male', 'female'}:
            target_gender = user_input.lower()
        elif user_input.lower() == 'm':
            target_gender = 'male'
        elif user_input.lower() == 'f':
            target_gender = 'female'
        else:
            print('Only male and female genders are available. Please try again')


    filename = input('Enter a filename (must be .wav): ')
    if not filename:
        break
    if not filename.endswith('.wav'):
        print('Not a .wav file, please try again')
        continue

    try:
        sr, _ = wavfile.read(filename)
        
        sr = 48000
        pitch, intonation = get_pitch_intonation(filename, sr)
        f1_diff, f2_diff = get_formant_diffs(filename, sr)
        resonance = f1_diff + f2_diff
    except Exception as e:
        print('Error:', e)
        continue

    sample = pd.DataFrame(data=[[pitch, intonation, resonance]], columns=['pitch', 'intonation', 'resonance'])
    print(sample)
    gender_pred = rf.predict(sample)[0]

    if target_gender != gender_pred:
        want_male = target_gender == 'male'
        shap_vals = explainer(sample)
        to_change = shap_vals.values[:,:,1].argmin() if want_male else shap_vals.values[:,1].argmax()

        print(f'You wanted to be perceived as {target_gender}, but your voice was classified as {gender_pred}')
        print('To improve the gender perception of your voice, we recommend that you ', end='')
        if to_change == 0:
            print(f'{"decrease" if want_male else "increase"} your pitch')
        elif to_change == 1:
            print(f'{"decrease" if want_male else "increase"} the variation of your pitch')
        else:
            print(f'{"darken" if want_male else "brighten"} your resonance')
    else:
        print('Your voice matches your gender. Good job!')
        
        
    print('\n' + '-'*20 + '\n')
    
    

Enter your target gender (male, female): m
Enter a filename (must be .wav): jack_audio1.wav
       pitch  intonation  resonance
0  93.733276   10.905282 -273.47093
Your voice matches your gender. Good job!

--------------------

Enter your target gender (male, female): f
Enter a filename (must be .wav): jack_audio2.wav
        pitch  intonation   resonance
0  109.170746   18.935408 -308.337344
You wanted to be perceived as female, but your voice was classified as male
To improve the gender perception of your voice, we recommend that you increase the variation of your pitch

--------------------

Enter your target gender (male, female): f
Enter a filename (must be .wav): adrienne_audio1.wav
        pitch  intonation   resonance
0  213.108173    36.99037 -143.270554
Your voice matches your gender. Good job!

--------------------

Enter your target gender (male, female): m
Enter a filename (must be .wav): adrienne_audio2.wav
        pitch  intonation  resonance
0  160.974106   14.525993 -